### from : rag_cot_peactice.md

## 환경 정보

In [1]:
# .env 로드 & LLM 초기화
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(
    model_name=os.getenv("OPENAI_LLM_MODEL"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0
)

## AnswerParser

In [3]:
# 가짜 Retriever (Top-3 문서 반환)
docs = {
    "doc1": "대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다.",
    "doc2": "서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다.",
    "doc3": "서울의 인구는 약 천만 명이며, 교통·문화 인프라가 잘 갖춰져 있습니다."
}
def fake_retriever(query: str, top_k: int = 3):
    return [docs[f"doc{i}"] for i in range(1, top_k+1)]

# OutputParser 정의
import re
from langchain.schema import BaseOutputParser

class FinalAnswerParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        pattern = r"3\.\s*\*\*?최종\s*답변\*{0,2}[:\s]*(.+)$"
        m = re.search(pattern, text, re.DOTALL | re.MULTILINE)
        if m:
            return m.group(1).strip()
        fb = re.search(r"최종\s*답변[:\s]*(.+)", text, re.DOTALL)
        return fb.group(1).strip() if fb else text.strip()

### 난이도 하 (Easy)

In [8]:
query = "서울의 수도는 어디인가요?"
retrieved = fake_retriever(query, top_k=2)
retrieved


['대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다.',
 '서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다.']

In [15]:
template_normal = """=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}
지시사항:
1. 핵심 정보 추출  
2. 정보를 조합  
3. **최종 답변**:
"""     # Normal Instructions 

template_cot = """=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}

[지시사항]
1. 핵심 정보 추출  
2. 정보를 조합  
3. **최종 답변**:
"""     # COT Instructions 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_easy = PromptTemplate(
    input_variables=["doc1", "doc2", "doc3", "question"],   # Contexts
    # template=template_normal,
    template=template_cot,
)

chain_easy = prompt_easy | llm | StrOutputParser()
# chain_easy = prompt_easy | llm | FinalAnswerParser()
result_easy = chain_easy.invoke({
    "doc1": retrieved[0],
    "doc2": retrieved[1],
    "doc3": retrieved[2] if len(retrieved) > 2 else "",
    "question": query
})
print("Easy 결과:", result_easy)

Easy 결과: 1. **핵심 정보 추출**  
   - 서울의 인구: 약 천만 명  
   - 서울의 대표적 관광지: 경복궁, 남산타워, 명동 등  
   - 교통·문화 인프라: 잘 갖춰져 있음  

2. **정보 조합**  
   서울은 약 천만 명의 인구를 가진 대도시로, 다양한 문화와 역사를 경험할 수 있는 관광지가 풍부합니다. 대표적인 관광지인 경복궁은 한국의 전통 문화를 체험할 수 있는 장소이며, 남산타워는 서울의 전경을 한눈에 볼 수 있는 명소입니다. 또한, 명동은 쇼핑과 먹거리를 즐길 수 있는 활기찬 거리로, 많은 관광객들이 찾는 곳입니다. 이러한 관광지들은 서울의 풍부한 문화적 자산을 반영하고 있습니다. 더불어, 서울은 교통과 문화 인프라가 잘 갖춰져 있어, 대중교통을 이용해 쉽게 이동할 수 있으며, 다양한 문화 행사와 프로그램이 진행되어 여행객들이 편리하게 즐길 수 있는 환경을 제공합니다.

3. **최종 답변**:  
서울은 약 천만 명의 인구가 거주하는 대도시로, 경복궁, 남산타워, 명동과 같은 다양한 관광지를 보유하고 있습니다. 이러한 관광지는 서울의 풍부한 역사와 문화를 체험할 수 있는 기회를 제공합니다. 또한, 잘 갖춰진 교통·문화 인프라 덕분에 여행객들은 편리하게 이동하며 다양한 문화적 경험을 즐길 수 있습니다. 따라서 서울은 여행하기에 매우 좋은 도시입니다.


### 난이도 중 (Medium)

In [10]:
query = "서울의 주요 관광지 3곳과 특징을 알려주세요."
retrieved = fake_retriever(query, top_k=3)
retrieved


['대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다.',
 '서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다.',
 '서울의 인구는 약 천만 명이며, 교통·문화 인프라가 잘 갖춰져 있습니다.']

In [11]:
prompt_med = PromptTemplate(
    input_variables=["doc1","doc2","doc3","question"],
    template="""문서들을 읽고 단계별로 답하세요.

=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}

[지시사항]
1) 각 문서에서 핵심 키워드 추출  
2) 관광지별 특징 정리  
3) **최종 답변**을 표 형태로 제시
"""
)

chain_med = prompt_med | llm | StrOutputParser()
# chain_med = prompt_med | llm | FinalAnswerParser()
result_med = chain_med.invoke({
    "doc1": retrieved[0],
    "doc2": retrieved[1],
    "doc3": retrieved[2] if len(retrieved) > 2 else "",
    "question": query
})
print("Medium 결과:", result_med)

Medium 결과: 1) 각 문서에서 핵심 키워드 추출  
- 문서1: 서울, 수도, 한강, 발달  
- 문서2: 관광지, 경복궁, 남산타워, 명동  
- 문서3: 인구, 천만 명, 교통, 문화 인프라  

2) 관광지별 특징 정리  
- 경복궁: 조선 왕조의 주요 궁궐로, 역사적 가치가 높고 아름다운 건축물과 정원이 있음.  
- 남산타워: 서울의 상징적인 전망대이며, 서울 전경을 한눈에 볼 수 있는 장소로 유명함.  
- 명동: 쇼핑과 음식 문화가 발달한 지역으로, 많은 관광객이 방문하는 번화가임.  

3) **최종 답변**  

| 관광지     | 특징                                               |
|------------|----------------------------------------------------|
| 경복궁     | 조선 왕조의 주요 궁궐, 역사적 가치와 아름다운 건축물 |
| 남산타워   | 서울의 상징적인 전망대, 서울 전경을 볼 수 있는 장소 |
| 명동       | 쇼핑과 음식 문화가 발달한 번화가, 많은 관광객 방문 |


### 난이도 상 (Hard)

In [12]:
query = "서울의 인구, 관광지, 교통 인프라를 종합해 여행하기 좋은 이유를 논리적으로 설명해주세요."
retrieved = fake_retriever(query, top_k=3)
retrieved


['대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다.',
 '서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다.',
 '서울의 인구는 약 천만 명이며, 교통·문화 인프라가 잘 갖춰져 있습니다.']

In [13]:
prompt_hard = PromptTemplate(
    input_variables=["doc1","doc2","doc3","question"],
    template="""아래 문서들을 바탕으로 심층 분석을 수행하세요.

=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}

[지시사항]
1. 문서별 핵심 데이터(인구·관광·교통) 정리  
2. 데이터 간 상호관계 분석  
3. 분석 기반 이유 서술  
4. **최종 답변**: 서론·본론·결론 구조로 제시
"""
)

chain_hard = prompt_hard | llm | StrOutputParser()
# chain_hard = prompt_hard | llm | FinalAnswerParser()
result_hard = chain_hard.invoke({
    "doc1": retrieved[0],
    "doc2": retrieved[1],
    "doc3": retrieved[2] if len(retrieved) > 2 else "",
    "question": query
})
print("Hard 결과:", result_hard)

Hard 결과: ### 1. 문서별 핵심 데이터 정리

- **문서1**: 
  - 수도: 서울
  - 위치: 한강을 끼고 발달한 도시

- **문서2**: 
  - 대표적 관광지: 경복궁, 남산타워, 명동 등

- **문서3**: 
  - 인구: 약 천만 명
  - 교통·문화 인프라: 잘 갖춰져 있음

### 2. 데이터 간 상호관계 분석

- **인구와 관광지**: 서울의 인구가 약 천만 명이라는 것은 도시가 활기차고 다양한 문화적 활동이 이루어짐을 의미한다. 많은 인구는 관광객들에게 다양한 경험을 제공할 수 있는 기반이 된다. 예를 들어, 경복궁과 같은 역사적 장소는 많은 방문객을 수용할 수 있는 인프라와 서비스가 필요하다.

- **관광지와 교통 인프라**: 서울의 대표 관광지들은 교통 인프라가 잘 갖춰져 있어 접근성이 뛰어나다. 예를 들어, 경복궁은 지하철과 버스 등 대중교통으로 쉽게 접근할 수 있으며, 남산타워와 명동 또한 대중교통으로 연결되어 있어 관광객들이 편리하게 이동할 수 있다.

- **인구와 교통 인프라**: 천만 명의 인구를 수용하기 위해 서울은 교통 인프라를 지속적으로 발전시켜왔다. 이는 관광객들이 서울을 방문할 때 편리하게 이동할 수 있도록 도와준다. 잘 발달된 교통망은 관광지 간의 이동을 원활하게 하여 여행의 효율성을 높인다.

### 3. 분석 기반 이유 서술

서울은 약 천만 명의 인구가 거주하는 대도시로, 다양한 문화와 역사를 경험할 수 있는 관광지가 풍부하다. 경복궁, 남산타워, 명동과 같은 대표적인 관광지는 서울의 역사적, 문화적 가치를 잘 보여준다. 이러한 관광지는 대중교통으로 쉽게 접근할 수 있어, 관광객들이 편리하게 여행할 수 있는 환경을 제공한다. 또한, 서울의 교통 인프라는 잘 갖춰져 있어, 관광객들이 여러 관광지를 효율적으로 방문할 수 있도록 돕는다. 

### 4. 최종 답변

**서론**  
서울은 대한민국의 수도로서, 한강을 끼고 발달한 도시입니다. 이 도시는 약 천만 명의 인구가 거주하며, 다양한 관광지와